In [1]:
import requests 
from bs4 import BeautifulSoup

In [21]:
# getting the data from the URL. 
page = requests.get("https://forecast.weather.gov/MapClick.php?lon=-118.5736&lat=37.28093")

#What is the soup variable storing? All the content on the page. 
soup = BeautifulSoup(page.content, 'html.parser')

#Then from within ALL the content,
#we want to find the div section with the id seven-day-forecast
# and get all text from within it. 
seven_day = soup.find(id="seven-day-forecast")

#When we look at the information within the seven day forecast id, 
#there's still a lot of information in there that we don't want. 
# Or in other words, we want to further filter the data we have. 

forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
# print(tonight.prettify())

In [22]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

Today
Mostly Sunny
High: 63 °F


In [23]:
img = tonight.find("img")
desc = img['title']

print(desc)

Today: Mostly sunny, with a high near 63. West northwest wind 7 to 13 mph. 


In [5]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Today',
 'Tonight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday']

In [6]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Mostly Sunny', 'Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny']
['High: 67 °F', 'Low: 55 °F', 'High: 69 °F', 'Low: 56 °F', 'High: 70 °F', 'Low: 56 °F', 'High: 71 °F', 'Low: 56 °F', 'High: 70 °F']
['Today: Mostly sunny, with a high near 67. Northwest wind 9 to 17 mph, with gusts as high as 22 mph. ', 'Tonight: Clear, with a low around 55. West northwest wind 12 to 17 mph decreasing to 6 to 11 mph in the evening. Winds could gust as high as 22 mph. ', 'Tuesday: Sunny, with a high near 69. Northwest wind 6 to 15 mph, with gusts as high as 18 mph. ', 'Tuesday Night: Mostly clear, with a low around 56. West northwest wind 10 to 14 mph. ', 'Wednesday: Sunny, with a high near 70. Light and variable wind becoming west 8 to 13 mph in the afternoon. ', 'Wednesday Night: Mostly clear, with a low around 56.', 'Thursday: Sunny, with a high near 71.', 'Thursday Night: Mostly clear, with a low around 56.', 'Friday: Sunny, with a high near 70.']


In [7]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,desc,period,short_desc,temp
0,"Today: Mostly sunny, with a high near 67. Nort...",Today,Mostly Sunny,High: 67 °F
1,"Tonight: Clear, with a low around 55. West nor...",Tonight,Clear,Low: 55 °F
2,"Tuesday: Sunny, with a high near 69. Northwest...",Tuesday,Sunny,High: 69 °F
3,"Tuesday Night: Mostly clear, with a low around...",TuesdayNight,Mostly Clear,Low: 56 °F
4,"Wednesday: Sunny, with a high near 70. Light a...",Wednesday,Sunny,High: 70 °F
5,"Wednesday Night: Mostly clear, with a low arou...",WednesdayNight,Mostly Clear,Low: 56 °F
6,"Thursday: Sunny, with a high near 71.",Thursday,Sunny,High: 71 °F
7,"Thursday Night: Mostly clear, with a low aroun...",ThursdayNight,Mostly Clear,Low: 56 °F
8,"Friday: Sunny, with a high near 70.",Friday,Sunny,High: 70 °F


In [19]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)",expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    67
1    55
2    69
3    56
4    70
5    56
6    71
7    56
8    70
Name: temp_num, dtype: object

In [9]:
weather["temp_num"].mean()


63.333333333333336

In [10]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [11]:
weather[is_night]


,desc,period,short_desc,temp,temp_num,is_night
1,"Tonight: Clear, with a low around 55. West nor...",Tonight,Clear,Low: 55 °F,55,True
3,"Tuesday Night: Mostly clear, with a low around...",TuesdayNight,Mostly Clear,Low: 56 °F,56,True
5,"Wednesday Night: Mostly clear, with a low arou...",WednesdayNight,Mostly Clear,Low: 56 °F,56,True
7,"Thursday Night: Mostly clear, with a low aroun...",ThursdayNight,Mostly Clear,Low: 56 °F,56,True
